In [179]:
import pandas as pd
import numpy as np

In [32]:
labels = pd.read_csv("labels_T.txt", header=None, sep="\t")
labels = labels[:-1] #remove last row
data = pd.read_csv("WBexpenditure_240519")

print(">> WB DATA \n",data.head())
print("\ndimensions: ", data.shape)
print("\n\n>> LABELS \n",labels.head())
print("\ndimensions: ", labels.shape)

>> WB DATA 
   Unnamed: 0  Unnamed: 1 country_code      country               sector  \
0    country           0          AFG  Afghanistan            Alsectors   
1    country           1          AFG  Afghanistan     Foodandbeverages   
2    country           2          AFG  Afghanistan  Clothingandfootwear   
3    country           3          AFG  Afghanistan              Housing   
4    country           4          AFG  Afghanistan               Energy   

  urbanity      all  lowest      low   middle  higher  
0    rural  1148.27  847.56  1625.97  3970.46     NaN  
1    rural   737.74  571.16  1005.54  1870.94     NaN  
2    rural    51.72   43.24    65.22   126.92     NaN  
3    rural    56.99   41.01    82.74   159.13     NaN  
4    rural    70.08   49.84   102.41   237.52     NaN  

dimensions:  (2366, 11)


>> LABELS 
              0    1           2                             3   4
0  Afghanistan  AFG  Industries                   Agriculture NaN
1  Afghanistan  AFG  Industri

In [114]:
# split data urban/rural
data_urban = data[data["urbanity"] == "urban"]
data_urban = data_urban[["country_code", "country", "sector", "all"]]
data_rural = data[data["urbanity"] == "rural"]
data_rural = data_rural[["country_code", "country", "sector", "all"]]
print(">> URBAN DATA \n",data_urban[:15])
print("\ndimensions: ", data_urban.shape)
print("\n\n>> RURAL DATA \n",data_rural.head())
print("\ndimensions: ", data_rural.shape)

>> URBAN DATA 
      country_code      country                 sector      all
1183          AFG  Afghanistan              Alsectors  1967.05
1184          AFG  Afghanistan       Foodandbeverages   904.50
1185          AFG  Afghanistan    Clothingandfootwear    83.10
1186          AFG  Afghanistan                Housing   328.54
1187          AFG  Afghanistan                 Energy   151.33
1188          AFG  Afghanistan              Transport   137.69
1189          AFG  Afghanistan                  Water     3.17
1190          AFG  Afghanistan        Educationsector     5.71
1191          AFG  Afghanistan                 Health    62.44
1192          AFG  Afghanistan                    ICT    53.72
1193          AFG  Afghanistan      Financialservices     0.00
1194          AFG  Afghanistan  Othergoodsandservices   199.90
1195          AFG  Afghanistan            Appliances.    36.93
1196          ALB      Albania              Alsectors  4021.85
1197          ALB      Albania       Fo

In [115]:
## RENAME WB CONSUMPTION CATEGORIES USING CONCORDANCE TABLE
data_urban = data_urban[data_urban.sector != "Alsectors"]
data_urban = data_urban.replace(to_replace ="Foodandbeverages", value ="Food & Beverages")
data_urban = data_urban.replace(to_replace ="Clothingandfootwear", value ="Textiles and Wearing Apparel")
data_urban = data_urban.replace(to_replace ="Housing", value ="Construction")
data_urban = data_urban.replace(to_replace ="Energy", value ="Electricity, Gas and Water")
data_urban = data_urban.replace(to_replace ="Transport", value ="Transport")
data_urban = data_urban.replace(to_replace ="Water", value ="Electricity, Gas and Water")
data_urban = data_urban.replace(to_replace ="Educationsector", value ="Education, Health and Other Services")
data_urban = data_urban.replace(to_replace ="Health", value ="Education, Health and Other Services")
data_urban = data_urban.replace(to_replace ="ICT", value ="Electrical and Machinery")
data_urban = data_urban.replace(to_replace ="Financialservices", value ="Finacial Intermediation and Business Activities")
data_urban = data_urban.replace(to_replace ="Othergoodsandservices", value ="Retail Trade")
data_urban = data_urban.replace(to_replace ="Appliances.", value ="Private Households") # appliances = personal care?

data_urban = data_urban[data_urban.sector != "Alsectors"]

data_urban

,country_code,country,sector,all
1184,AFG,Afghanistan,Food & Beverages,904.50
1185,AFG,Afghanistan,Textiles and Wearing Apparel,83.10
1186,AFG,Afghanistan,Construction,328.54
1187,AFG,Afghanistan,"Electricity, Gas and Water",151.33
1188,AFG,Afghanistan,Transport,137.69
...,...,...,...,...
2361,ZMB,Zambia,"Education, Health and Other Services",2.39
2362,ZMB,Zambia,Electrical and Machinery,37.83
2363,ZMB,Zambia,Finacial Intermediation and Business Activities,0.00
2364,ZMB,Zambia,Retail Trade,79.38


In [116]:
# sum repeated WB categories to match EORA
all_urban = data_urban.groupby(['country_code', 'sector'])['all'].sum()
all_urban = pd.DataFrame(all_urban)
all_urban.index.name = 'sector'
all_urban.reset_index(inplace=True)
data_urban = all_urban
data_urban

,country_code,sector,all
0,AFG,Construction,328.54
1,AFG,"Education, Health and Other Services",68.15
2,AFG,Electrical and Machinery,53.72
3,AFG,"Electricity, Gas and Water",154.50
4,AFG,Finacial Intermediation and Business Activities,0.00
...,...,...,...
905,ZMB,Food & Beverages,395.08
906,ZMB,Private Households,15.19
907,ZMB,Retail Trade,79.38
908,ZMB,Textiles and Wearing Apparel,28.47


In [172]:
df_urban = pd.DataFrame(labels[[1, 3]])
# df_urban = df_urban.reset_index()
df_urban.columns = ['country_code', 'category']
df_urban["consumption"] = 0
df_urban

,country_code,category,consumption
0,AFG,Agriculture,0
1,AFG,Fishing,0
2,AFG,Mining and Quarrying,0
3,AFG,Food & Beverages,0
4,AFG,Textiles and Wearing Apparel,0
...,...,...,...
4909,ZWE,Public Administration,0
4910,ZWE,"Education, Health and Other Services",0
4911,ZWE,Private Households,0
4912,ZWE,Others,0


In [173]:
for idx, row in df_urban.iterrows():
#     i = df_urban.get_value(idx, 'country_code')
#     j = df_urban.get_value(idx, 'category')
#     k = df_urban.get_value(idx, 'consumption')
#     for idy in data_urban.index:
#         m = data_urban.get_value(idy, 'country_code')
#         n = data_urban.get_value(idy, 'sector')
#         p = data_urban.get_value(idy, 'all')
#         if (i == m and j == n):
#             df_urban.set_value(idx, 'consumption', p)  
    for idy, row in data_urban.iterrows():
        if (df_urban.loc[idx,'country_code'] == data_urban.loc[idy,'country_code'] and df_urban.loc[idx,'category'] == data_urban.loc[idy,'sector']):
            df_urban.loc[idx,'consumption']  = data_urban.loc[idy, 'all'] 

#     for i, j, k in zip(df_urban["country_code"], df_urban["category"], df_urban["consumption"]):
#         for m, n, p in zip(data_urban["country_code"], data_urban["sector"], data_urban["all"]):
#             if (i == m and j == n):
#                 df_urban.at[k, "consumption"] = p
#                 print(i,j,k)

In [174]:
df_urban[:30]

,country_code,category,consumption
0,AFG,Agriculture,0.00
1,AFG,Fishing,0.00
2,AFG,Mining and Quarrying,0.00
3,AFG,Food & Beverages,904.50
4,AFG,Textiles and Wearing Apparel,83.10
5,AFG,Wood and Paper,0.00
6,AFG,"Petroleum, Chemical and Non-Metallic Mineral P...",0.00
7,AFG,Metal Products,0.00
8,AFG,Electrical and Machinery,53.72
9,AFG,Transport Equipment,0.00


In [175]:
## REPEAT PROCESS FOR RURAL CONSUMPTION ##
data_rural = data_rural[data_rural.sector != "Alsectors"]
data_rural = data_rural.replace(to_replace ="Foodandbeverages", value ="Food & Beverages")
data_rural = data_rural.replace(to_replace ="Clothingandfootwear", value ="Textiles and Wearing Apparel")
data_rural = data_rural.replace(to_replace ="Housing", value ="Construction")
data_rural = data_rural.replace(to_replace ="Energy", value ="Electricity, Gas and Water")
data_rural = data_rural.replace(to_replace ="Transport", value ="Transport")
data_rural = data_rural.replace(to_replace ="Water", value ="Electricity, Gas and Water")
data_rural = data_rural.replace(to_replace ="Educationsector", value ="Education, Health and Other Services")
data_rural = data_rural.replace(to_replace ="Health", value ="Education, Health and Other Services")
data_rural = data_rural.replace(to_replace ="ICT", value ="Electrical and Machinery")
data_rural = data_rural.replace(to_replace ="Financialservices", value ="Finacial Intermediation and Business Activities")
data_rural = data_rural.replace(to_replace ="Othergoodsandservices", value ="Retail Trade")
data_rural = data_rural.replace(to_replace ="Appliances.", value ="Private Households") # appliances = personal care?

data_rural = data_rural[data_rural.sector != "Alsectors"]

all_rural = data_rural.groupby(['country_code', 'sector'])['all'].sum()
all_rural = pd.DataFrame(all_rural)
all_rural.index.name = 'sector'
all_rural.reset_index(inplace=True)
data_rural = all_rural

df_rural = pd.DataFrame(labels[[1, 3]])
# df_rural = df_rural.reset_index()
df_rural.columns = ['country_code', 'category']
df_rural["consumption"] = 0
df_rural

,country_code,category,consumption
0,AFG,Agriculture,0
1,AFG,Fishing,0
2,AFG,Mining and Quarrying,0
3,AFG,Food & Beverages,0
4,AFG,Textiles and Wearing Apparel,0
...,...,...,...
4909,ZWE,Public Administration,0
4910,ZWE,"Education, Health and Other Services",0
4911,ZWE,Private Households,0
4912,ZWE,Others,0


In [176]:
for idx, row in df_rural.iterrows(): 
    for idy, row in data_rural.iterrows():
        if (df_rural.loc[idx,'country_code'] == data_rural.loc[idy,'country_code'] and df_rural.loc[idx,'category'] == data_rural.loc[idy,'sector']):
            df_rural.loc[idx,'consumption']  = data_rural.loc[idy, 'all'] 

df_rural[:30]

,country_code,category,consumption
0,AFG,Agriculture,0.00
1,AFG,Fishing,0.00
2,AFG,Mining and Quarrying,0.00
3,AFG,Food & Beverages,737.74
4,AFG,Textiles and Wearing Apparel,51.72
5,AFG,Wood and Paper,0.00
6,AFG,"Petroleum, Chemical and Non-Metallic Mineral P...",0.00
7,AFG,Metal Products,0.00
8,AFG,Electrical and Machinery,11.28
9,AFG,Transport Equipment,0.00


In [186]:
df_urban_diag = pd.DataFrame(np.diag(df_urban["consumption"]))
df_rural_diag = pd.DataFrame(np.diag(df_rural["consumption"]))
print(df_urban_diag.shape)
print(df_urban_diag)

(4914, 4914)
      0     1     2      3     4     5     6     7     8     9     ...  4904  \
0      0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1      0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2      0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3      0.0   0.0   0.0  904.5   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4      0.0   0.0   0.0    0.0  83.1   0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...    ...   ...   ...   ...   ...   ...   ...  ...   ...   
4909   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4910   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4911   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4912   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4913   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      4905  4906  4907  49

In [187]:
# output to csv
df_urban.to_csv("wbconsumption_urban_eora.csv") # 4919x1 urban
df_rural.to_csv("wbconsumption_rural_eora.csv") # 4919x1 rural
df_urban_diag.to_csv("wbconsumption_urban_eora_diag.csv") # 4919x4919 urban
df_rural_diag.to_csv("wbconsumption_rural_eora_diag.csv") # 4919x4919 rural